In [83]:
from random import seed
from random import random
from math import exp
import pandas as pd
from sklearn.datasets import load_iris
import numpy as np
from sklearn.preprocessing import OneHotEncoder


In [90]:
class Network:
  #constructor
  def __init__(self, n_inputs, n_hidden, n_outputs=3, bias=1):
    self.n_inputs = n_inputs # number of input unit
    self.n_hidden = n_hidden # number of hidden unit
    self.n_outputs = n_outputs # number of output unit
    self.bias = bias # bias parameter
    
    # parameters of weight on input to hidden layer 
    self.weights_ItoH = np.random.uniform(-1, 1, (n_inputs+1, n_hidden)) 
    self.dweights_ItoH = np.zeros((n_inputs+1, n_hidden))
    
    # parameters of weight on hidden to output layer 
    self.weights_HtoO = np.random.uniform(-1, 1, (n_hidden+1, n_outputs))
    self.dweights_HtoO = np.zeros((n_hidden+1, n_outputs))
    
    # output value and error of hidden layer
    self.pre_activation_H = np.zeros(n_hidden)
    self.post_activation_H = np.zeros(n_hidden)
    self.error_H = np.zeros(n_hidden)
    
    # output value and error of output layer
    self.pre_activation_O = np.zeros(n_outputs)
    self.post_activation_O = np.zeros(n_outputs)
    self.error_O = np.zeros(n_outputs)
  
  # Net calculation method
  ## Calculate net for an input
  def calculate_net_ItoH(self, sample, node):
    input_plus_bias = np.append(self.data[sample,:], self.bias)
    return np.dot(input_plus_bias, self.weights_ItoH[:, node])
  ## Calculate net for a hidden unit
  def calculate_net_HtoO(self, node):
    hidden_plus_bias =  np.append(self.post_activation_H, self.bias)
    return np.dot(hidden_plus_bias, self.weights_HtoO[:, node])

  # activation function
  def activation(self, x):
  	return 1.0/(1.0 + np.exp(-x))

  def one_hot_encode(self, target):
    encoder = OneHotEncoder(sparse=False)
    new_target = target.reshape(len(target), 1)
    target_encode = encoder.fit_transform(new_target)
    return target_encode

  #fit the network to the data
  def fit(self, data, target, epoch_limit=100, mini_batch_limit=10):
    self.data = data
    self.target = self.one_hot_encode(target)
    self.epoch_limit = epoch_limit

    len_data = len(data)

    # iterate each epoch
    for epoch in range(epoch_limit):
      
      #iterate each instance
      mini_batch_count = 0
      for instance in range(len_data):
            
        # From input layer to hidden layer
        ## iterate every hidden layer to fill the values
        for hidden_unit in range(self.n_hidden):
          ### calculate the net input
          self.pre_activation_H[hidden_unit] = self.calculate_net_ItoH(instance, hidden_unit)
          ### calculate the activated value
          self.post_activation_H[hidden_unit] = self.activation(self.pre_activation_H[hidden_unit])

        # From hidden layer to output layer
        for output_unit in range(self.n_outputs):
          ### calculate the net input
          self.pre_activation_O[output_unit] = self.calculate_net_HtoO(output_unit)
          ### calculate the activated value
          self.post_activation_O[output_unit] = self.activation(self.pre_activation_O[output_unit])
      
        #for debug
        if(epoch_limit == 1):
            print('INSTANCE:', instance )
            print('WEIGHTS\n', self.weights_ItoH, '\n', self.weights_HtoO)
            print('OUTPUTS\n', self.post_activation_H, '\n', self.post_activation_O)

        # Backpropagation
        ## if already at minibatch limit or at the last instance, update the weight 
        if((mini_batch_count == mini_batch_limit) or (instance == len_data - 1)):
          #update weight - hidden to output
          self.weights_HtoO = np.add(self.weights_HtoO, self.dweights_HtoO)
          
          #update weight - input to hidden
          self.weights_ItoH = np.add(self.weights_ItoH, self.dweights_ItoH)

          mini_batch_count = 0
        
        ## if below minibatch limit, update delta-weight
        else:
          ### update delta-weight from output
          for hidden_unit in range(self.n_hidden + 1): # (+1 accomodating bias)
            for output_unit in range(self.n_outputs):
              #### (Minus sign merged). Formula: (target_ok - out_ok) * out_ok * (1 - out_ok) * out_hj
              target_o = self.target[instance][output_unit]
              out_o = self.post_activation_O[output_unit]
              
              ##### calculating weight of bias
              if (hidden_unit == self.n_hidden): 
                out_h = self.bias
              ##### calculating weight of activated hidden unit
              else:
                out_h = self.post_activation_H[hidden_unit]

              self.error_O[output_unit] = (target_o - out_o) * out_o * (1 - out_o)
              self.dweights_HtoO[hidden_unit][output_unit] += self.error_O[output_unit] * out_h

          ### update delta-weight from hidden layer
          for input_unit in range(self.n_inputs + 1): # (+1 accomodating bias)
            for hidden_unit in range(self.n_hidden):
              #### Formula: sigma_ok(error_o * w_ho) * out_hj * (1 - out_hj) * input_i
              sigma_err_output = np.dot(self.error_O, self.weights_HtoO[hidden_unit,:])
              out_h = self.post_activation_H[hidden_unit]

              ##### calculating weight of bias
              if(input_unit == self.n_inputs): 
                input_i = self.bias
              ##### calculating weight of input unit
              else:
                input_i = self.data[instance, input_unit] 
              
              self.error_H[hidden_unit] = sigma_err_output * out_h * (1 - out_h)
              self.dweights_ItoH[input_unit][hidden_unit] += self.error_H[hidden_unit] * input_i

          mini_batch_count += 1
        

  def predict(self, data):
    self.data = data
    result = np.zeros(len(data))
    #iterate each instance
    for instance in range(len(data)):      
      ## iterate every hidden layer to fill the values
      for hidden_unit in range(self.n_hidden):
        ### calculate the net input
        self.pre_activation_H[hidden_unit] = self.calculate_net_ItoH(instance, hidden_unit)
        ### calculate the activated value
        self.post_activation_H[hidden_unit] = self.activation(self.pre_activation_H[hidden_unit])

      max_value = 0
      max_index = -1 
      # From hidden layer to output layer
      for output_unit in range(self.n_outputs):
        ### calculate the net input
        self.pre_activation_O[output_unit] = self.calculate_net_HtoO(output_unit)
        ### calculate the activated value
        self.post_activation_O[output_unit] = self.activation(self.pre_activation_O[output_unit])
        if(self.post_activation_O[output_unit] >= max_value ):
          max_value = self.post_activation_O[output_unit]
          max_index = output_unit
      print(self.post_activation_O)
      print('instance no:', instance, 'prediction result:', max_index)
      result = np.append(result, max_index)
    
    return result
    

In [92]:
# Training
print('Data Iris')
load, target = load_iris(return_X_y=True)
iris_data = pd.DataFrame(load, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_data['label'] = pd.Series(target)

shuffled_data = iris_data.sample(frac=1)
train_X = shuffled_data.drop('label',axis=1,inplace=False).values
train_y = shuffled_data['label'].values

net = Network(4, 4)
net.fit(train_X, train_y, epoch_limit=10)

#Testing
shuffled_data = iris_data.sample(n=20)
test_X = shuffled_data.drop('label',axis=1,inplace=False).values
test_y = shuffled_data['label'].values

result = net.predict(test_X)
print("Pred Result\n", result, sep='')
print("Original Data\n", test_y, sep='')

In [6]:
o = np.random.uniform(1, 10, 3)
j = np.random.uniform(1, 10, 3)
print(o, j)
np.dot(o, j)

[3.11740136 8.27094941 5.9164251 ] [3.51638315 5.36092207 1.32702712]


63.1531494133005

In [10]:
a = np.random.uniform(1,1,(3,5))
b = np.random.uniform(1,10,(3,5))

In [11]:
print(a, '\n')
print(b, '\n')
print(np.add(a,b))

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]] 

[[4.34001615 8.84582444 5.36002125 8.67189147 5.18367231]
 [8.42507471 4.31171515 7.97301027 6.15631649 9.27238015]
 [7.58748903 6.40231917 8.78151948 7.05457473 4.83981053]] 

[[ 5.34001615  9.84582444  6.36002125  9.67189147  6.18367231]
 [ 9.42507471  5.31171515  8.97301027  7.15631649 10.27238015]
 [ 8.58748903  7.40231917  9.78151948  8.05457473  5.83981053]]


In [15]:
a = np.random.uniform(1,1,5)
print(a)
a = np.append(a, 5)
print(a)

[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 5.]
